In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
from holoviews import opts
import panel as pn
pn.extension()
hv.extension('bokeh')
import geopandas as gpd
from shapely.geometry import Point
import folium
import branca.colormap as cm



In [ ]:
df=pd.read_csv('data/commute/CSVLog_20240802_141351.csv')
df.columns = df.columns.str.strip()
df.head()

In [ ]:
column_list=df.columns.tolist()
print(column_list)

In [ ]:
df=df[['Time (sec)', 'Vehicle speed (MPH)', 'Relative accelerator pedal position (%)','Engine RPM (RPM)', 'Boost (psi)', 'Engine Power (hp)', 'Mass air flow rate (lb/min)','Intake manifold absolute pressure (inHg)', 'Ambient air temperature (°F)','Trip Distance (miles)','Instant fuel economy (MPG)', 'Trip Fuel Economy (MPG)', 'Latitude (deg)', 'Longitude (deg)','Bearing (deg)', 'Hard Brake Count', 'Relative throttle position (%)']]
df.head()

In [ ]:
# To avoid math errors in situations where values = 0
df['Boost (psi)'] = df['Boost (psi)'].bfill().infer_objects()
df.loc[df['Boost (psi)'] == 0, 'Boost (psi)'] = 0.01

df.head()


In [ ]:
def clean_data(df):
    # Round columns 'Time (sec)', ' Engine RPM (RPM)' and 2 other columns (Number of decimals: 0)
    df = df.round({'Time (sec)': 0, 'Engine RPM (RPM)': 0, 'Engine Power (hp)': 0, 'Time (sec)': 0})
    # Round columns ' Instant fuel economy (MPG)', ' Vehicle speed (MPH)' and 4 other columns (Number of decimals: 1)
    df = df.round({'Instant fuel economy (MPG)': 1, 'Vehicle speed (MPH)': 1,'Relative throttle position (%)': 1, 'Relative accelerator pedal position (%)': 1, 'Trip Distance (miles)': 1, 'Trip Fuel Economy (MPG)': 1,'Boost (psi)':1})
    # Round columns ' Intake manifold absolute pressure (inHg)', ' Mass air flow rate (lb/min)', ' Bearing (deg)' (Number of decimals: 2)
    df = df.round({'Intake manifold absolute pressure (inHg)': 2, 'Mass air flow rate (lb/min)': 2,'Bearing (deg)': 2})
    # Drop columns: ' Intake manifold absolute pressure (inHg)', ' Mass air flow rate (lb/min)' and 3 other columns
    df = df.drop(columns=['Hard Brake Count','Relative throttle position (%)','Fuel type'])
    # Rename column 'Latitude (deg)' to 'latitude'
    df = df.rename(columns={'Latitude (deg)': 'latitude'})
    # Rename column 'Longitude (deg)' to 'longitude'
    df = df.rename(columns={'Longitude (deg)': 'longitude'})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

In [ ]:
#create dataframe where vehicle is stationary
df_stationary = df_clean[df_clean['Vehicle speed (MPH)'] == 0]
df_stationary.head()

In [ ]:
#creates dataframe where the vehicle is moving or is about to begin motion after being stationary
df_clean['next_mph'] = df_clean['Vehicle speed (MPH)'].shift(-1)
mask = (df_clean['Vehicle speed (MPH)'] > 0) | ((df_clean['Vehicle speed (MPH)'] == 0) & (df_clean['next_mph'] > 0))
df_moving= df_clean[mask].drop(columns=['next_mph'])
df_moving.head()

In [ ]:
#experimenting 
df_moving['Mass Air/Boost']=df_moving['Mass air flow rate (lb/min)']/df_moving['Boost (psi)']
df_moving['Mass Air/manPress']=df_moving['Mass air flow rate (lb/min)']/df_moving['Intake manifold absolute pressure (inHg)']
df_moving.head()

In [ ]:
#creates dataframe where vehicle is moving and speed is less than 10mph
mask = (df_moving['Vehicle speed (MPH)'] <= 10)
df_less_10 = df_moving[mask]
#df_less_10.head()
df_less_10.describe()

In [ ]:
#creates dataframe where vehicle is moving and speed is between 10 and 20mph
mask = (df_moving['Vehicle speed (MPH)'] >= 10) & (df_moving['Vehicle speed (MPH)'] <= 20)
df_10_20 = df_moving[mask]
#df_10_20.head()
df_10_20.describe()

In [ ]:
#creates dataframe where vehicle is moving and speed is between 20 and 30mph
mask = (df_moving['Vehicle speed (MPH)'] >= 20) & (df_moving['Vehicle speed (MPH)'] <= 30)
df_20_30 = df_moving[mask]
#df_20_30.head()
df_20_30.describe()

In [ ]:
#creates dataframe where vehicle is moving and speed is between 30 and 40mph
mask = (df_moving['Vehicle speed (MPH)'] >= 30) & (df_moving['Vehicle speed (MPH)'] <= 40)
df_30_40 = df_moving[mask]
#df_30_40.head()
df_30_40.describe()

In [ ]:
#creates dataframe where vehicle is moving and speed is between 40 and 50mph
mask = (df_moving['Vehicle speed (MPH)'] >= 40) & (df_moving['Vehicle speed (MPH)'] <= 50)
df_40_50 = df_moving[mask]
#df_40_50.head()
df_40_50.describe()

In [ ]:
mask = (df_moving['Vehicle speed (MPH)'] >= 50) & (df_moving['Vehicle speed (MPH)'] <= 60)
df_50_60 = df_moving[mask]
df_50_60.describe()

In [ ]:
#creates dataframe where vehicle is moving and speed is between 40 and 50mph
mask = (df_moving['Vehicle speed (MPH)'] >= 60) & (df_moving['Vehicle speed (MPH)'] <= 70)
df_60_70 = df_moving[mask]
#df_60_70.head()
df_60_70.describe()

In [ ]:
mask = (df_moving['Vehicle speed (MPH)'] >= 70)
df_grt_70 = df_moving[mask]
#df_grt_70.head()
df_grt_70.describe()

In [ ]:
mask = df_20_30['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

In [ ]:
mask = df_30_40['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

In [ ]:
mask = df_40_50['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

In [ ]:
mask = df_60_70['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

In [ ]:
mask = df_60_70['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

In [ ]:
mask = (df_moving['Vehicle speed (MPH)'] >= 70)
df_grt_70 = df_moving[mask]
df_grt_70.head()

In [ ]:

df_less_10_geo=df_less_10.copy()
df_less_10_geo.head()

df_10_20_geo = df_10_20.copy()
df_10_20_geo.head()

df_20_30_geo = df_20_30.copy()
df_20_30_geo.head()

df_30_40_geo = df_30_40.copy()
df_30_40_geo.head()

df_40_50_geo = df_40_50.copy()
df_40_50_geo.head()

df_50_60_geo = df_50_60.copy()


df_60_70_geo = df_60_70.copy()





In [ ]:
map_center = [df_60_70_geo['latitude'].mean(), df_60_70_geo['longitude'].mean()]
print(map_center)

In [ ]:
print("Available colormaps:")
for name in dir(cm):
    if 'colormap' in name.lower():
        print(name)

In [ ]:
map_center = [44.980164, -93.185692]
m = folium.Map(location=map_center, zoom_start=11)

#colormap = cm.linear.YlGnBu.scale(df_moving['Vehicle speed (MPH)'].min(), df_moving['Vehicle speed (MPH)'].max())
colormap = cm.LinearColormap(
    colors=['blue', 'green', 'yellow', 'red'],  # Custom color range
    vmin=df_moving['Vehicle speed (MPH)'].min(),
    vmax=df_moving['Vehicle speed (MPH)'].max(),
    caption='Vehicle Speed (MPH)'
)
colormap.add_to(m)

coordinates = df_moving[['latitude', 'longitude', 'Vehicle speed (MPH)']].values.tolist()
sorted_coordinates = sorted(coordinates, key=lambda x: x[2])

for i in range(len(df_moving) - 1):
    # Coordinates of the current segment
    start = [df_moving.iloc[i]['latitude'], df_moving.iloc[i]['longitude']]
    end = [df_moving.iloc[i + 1]['latitude'], df_moving.iloc[i + 1]['longitude']]
    
    # Value for color mapping (use average or a specific value)
    segment_value = (df_moving.iloc[i]['Vehicle speed (MPH)'] + df_moving.iloc[i + 1]['Vehicle speed (MPH)']) / 2
    
    # Create and add a PolyLine for each segment
    folium.PolyLine(
        locations=[start, end],
        color=colormap(segment_value),  # Map value to color
        weight=5,
        opacity=1
    ).add_to(m)
# Save the map to an HTML file
m.save('df_moving_interactive_map.html')


In [ ]:
map_center = [44.980164, -93.185692]
m = folium.Map(location=map_center, zoom_start=11)


# Prepare the list of coordinates for the PolyLine
coordinates = df_60_70_geo[['latitude', 'longitude']].values.tolist()

# Add a PolyLine to the map
folium.PolyLine(
    locations=coordinates,
    popup=[str(row['Time (sec)']) for idx, row in df_60_70_geo.iterrows()],
    weight=5,
    opacity=1,
    color='blue'
).add_to(m)

# Save the map to an HTML file
m.save('60_70_map.html')

In [ ]:
hvexplorer = df.hvplot.explorer()
hvexplorer

In [ ]:
df['Engine RPM (RPM)']=df['Engine RPM (RPM)']/100
df['Engine Power (hp)']=df['Engine Power (hp)']/10
df['Ambient air temperature (°F)']=df['Ambient air temperature (°F)']/10
df['Intake air temperature bank 1  sensor 1 (°F)']=df['Intake air temperature bank 1  sensor 1 (°F)']/10
#df['Relative throttle position (%)']=df['Relative throttle position (%)']/10
#df[' Boost (psi)']=df[' Boost (psi)']/10
#df[' Mass air flow rate (lb/min)']=df[' Mass air flow rate (lb/min)']/10


#df['Boost (psi)']= df['Boost (psi)']/10


In [ ]:
start_index = int(round(2227.003))  # or use int(2227.003)
end_index = int(round(2231.996))    # or use int(2255.994)

# Select rows between start_index and end_index (inclusive)
boosted = df.iloc[start_index:end_index + 1]

print(boosted.head())

In [ ]:
#boosted=df.iloc[2227:2233]
#boosted.head

In [ ]:
boosted.hvplot(
    kind='line',
    x='Time (sec)',
    y=['Engine RPM (RPM)','Boost (psi)','Relative throttle position (%)','Vehicle speed (MPH)' ], #'Engine Power (hp)'
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
df.hvplot(
    kind='line',
    x='Time (sec)',
    y=['Mass Air/Boost'],
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
df.hvplot(
    kind='line',
    x='Time (sec)',
    y=['Mass Air/manPress'],
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
df.hvplot(
    kind='line',
    x='Time (sec)',
    y1='Intake air temperature bank 1  sensor 1 (°F)',
    y2='Boost (psi)',
    legend='bottom_right',
    widget_location='bottom',
)